⚡ Intermédiaire | ⏱ 45 min | 🔑 Concepts : pdb, breakpoint, pas à pas, post-mortem

# 02 - Debugging en Python

## Objectifs

- Maîtriser les techniques de debugging modernes
- Utiliser **pdb** (Python Debugger) efficacement
- Comprendre **breakpoint()** et le debugging interactif
- Apprendre le post-mortem debugging
- Intégrer le debugging dans VS Code et Jupyter

## Prérequis

- Python 3.7+ (pour `breakpoint()`)
- Connaissance de base de Python
- Compréhension des exceptions

## 1. Print Debugging : Simple mais limité

La méthode la plus courante pour déboguer... mais pas la plus efficace.

In [ ]:
def calculer_factorielle(n):
    print(f"DEBUG: n = {n}")  # Print debugging
    if n < 0:
        raise ValueError("n doit être positif")
    if n == 0:
        return 1
    result = n * calculer_factorielle(n - 1)
    print(f"DEBUG: result pour n={n} : {result}")
    return result

calculer_factorielle(5)

### Limites du print debugging

- ❌ Pollue le code et les logs
- ❌ Difficile de voir l'état complet du programme
- ❌ Nécessite de modifier et relancer le code
- ❌ Oublier de retirer les `print()` avant le commit

**Solution** : Utiliser un vrai debugger !

## 2. breakpoint() : Le debugger intégré (Python 3.7+)

`breakpoint()` est la façon moderne de lancer le debugger Python.

**Avantages** :
- Pas besoin d'importer `pdb`
- Peut être désactivé avec la variable d'environnement `PYTHONBREAKPOINT=0`
- Configurable pour utiliser d'autres debuggers (ipdb, pudb, etc.)

In [ ]:
def diviser(a, b):
    """Division avec debugging."""
    breakpoint()  # Le debugger s'arrête ici
    resultat = a / b
    return resultat

# Note: breakpoint() ne fonctionne pas dans Jupyter par défaut
# Utilisez %debug magic à la place (voir section 7)

## 3. PDB : Commandes essentielles

Une fois dans le debugger, voici les commandes les plus utiles :

### Commandes de navigation

| Commande | Raccourci | Description |
|----------|-----------|-------------|
| `next` | `n` | Exécuter la ligne suivante (ne rentre pas dans les fonctions) |
| `step` | `s` | Exécuter la ligne suivante (rentre dans les fonctions) |
| `continue` | `c` | Continuer jusqu'au prochain breakpoint |
| `return` | `r` | Continuer jusqu'au return de la fonction actuelle |
| `quit` | `q` | Quitter le debugger |

### Commandes d'inspection

| Commande | Raccourci | Description |
|----------|-----------|-------------|
| `list` | `l` | Afficher le code autour de la ligne actuelle |
| `where` | `w` | Afficher la stack trace |
| `print` | `p` | Afficher la valeur d'une variable |
| `pp` | - | Pretty-print d'une variable |
| `args` | `a` | Afficher les arguments de la fonction actuelle |
| `whatis` | - | Afficher le type d'une variable |

### Commandes avancées

| Commande | Description |
|----------|-------------|
| `up` | Remonter d'un niveau dans la stack |
| `down` | Descendre d'un niveau dans la stack |
| `break` | Définir un nouveau breakpoint |
| `clear` | Supprimer un breakpoint |
| `!<statement>` | Exécuter du code Python arbitraire |

## 4. Exemple complet de debugging avec pdb

In [ ]:
%%writefile debug_example.py
#!/usr/bin/env python3
"""Exemple de debugging avec pdb."""

def calculer_moyenne(nombres):
    """Calcule la moyenne d'une liste de nombres."""
    total = sum(nombres)
    moyenne = total / len(nombres)
    return moyenne

def traiter_donnees(data):
    """Traite une liste de données."""
    resultats = []
    for item in data:
        breakpoint()  # Point d'arrêt ici
        if isinstance(item, list):
            moyenne = calculer_moyenne(item)
            resultats.append(moyenne)
        else:
            resultats.append(item)
    return resultats

if __name__ == "__main__":
    donnees = [[1, 2, 3], [4, 5, 6], 10, [7, 8, 9]]
    resultats = traiter_donnees(donnees)
    print(f"Résultats: {resultats}")

In [ ]:
# Pour exécuter avec le debugger :
# python debug_example.py

# Commandes à essayer dans pdb :
# l          # Voir le code
# p item     # Afficher la valeur de item
# p data     # Afficher toutes les données
# n          # Ligne suivante
# s          # Rentrer dans calculer_moyenne
# w          # Voir la stack
# c          # Continuer jusqu'au prochain breakpoint

print("Exécutez le script dans un terminal pour tester pdb")

## 5. Breakpoints conditionnels

Arrêter le debugger seulement quand une condition est vraie.

In [ ]:
%%writefile conditional_breakpoint.py
#!/usr/bin/env python3
"""Breakpoints conditionnels."""

def traiter_liste(elements):
    for i, element in enumerate(elements):
        # S'arrêter seulement quand l'élément est négatif
        if element < 0:
            breakpoint()
        
        resultat = element * 2
        print(f"Élément {i}: {element} -> {resultat}")

if __name__ == "__main__":
    data = [1, 5, -3, 10, -7, 20]
    traiter_liste(data)

Vous pouvez aussi définir des breakpoints conditionnels directement dans pdb :

```python
(Pdb) break 10, element < 0
# S'arrête à la ligne 10 seulement si element < 0
```

## 6. Post-mortem Debugging

Déboguer **après** qu'une exception s'est produite, sans avoir à relancer le programme.

In [ ]:
import pdb

def fonction_buggee():
    x = 10
    y = 0
    resultat = x / y  # ZeroDivisionError
    return resultat

try:
    fonction_buggee()
except Exception:
    # Lancer le debugger post-mortem
    # pdb.post_mortem()  # Décommentez pour tester
    print("Erreur détectée ! Utilisez pdb.post_mortem() pour déboguer")

In [ ]:
%%writefile postmortem_example.py
#!/usr/bin/env python3
"""Post-mortem debugging."""
import pdb
import sys

def diviser(a, b):
    return a / b

def main():
    try:
        resultat = diviser(10, 0)
        print(resultat)
    except Exception:
        # Lancer le debugger après l'exception
        type, value, traceback = sys.exc_info()
        pdb.post_mortem(traceback)

if __name__ == "__main__":
    main()

En ligne de commande, vous pouvez aussi utiliser :

```bash
python -m pdb -c continue script.py
# Lance pdb en mode post-mortem automatique
```

## 7. Debugging dans Jupyter

Jupyter a des commandes magiques spéciales pour le debugging.

In [ ]:
def fonction_avec_bug(liste):
    total = 0
    for i in range(len(liste) + 1):  # Bug : +1 cause un IndexError
        total += liste[i]
    return total

# Cette cellule va générer une erreur
try:
    fonction_avec_bug([1, 2, 3])
except IndexError as e:
    print(f"Erreur: {e}")
    print("Exécutez %debug dans la cellule suivante pour déboguer")

In [ ]:
# Décommentez et exécutez pour lancer le debugger post-mortem
# %debug

### Autres commandes magiques utiles

```python
%pdb on   # Active le debugger automatique sur les exceptions
%pdb off  # Désactive le debugger automatique

%%debug   # Exécute toute la cellule en mode debug
```

## 8. Debugging dans VS Code

VS Code offre un excellent debugger visuel intégré.

### Configuration : launch.json

Créez `.vscode/launch.json` :

```json
{
    "version": "0.2.0",
    "configurations": [
        {
            "name": "Python: Current File",
            "type": "python",
            "request": "launch",
            "program": "${file}",
            "console": "integratedTerminal",
            "justMyCode": false  // Pour déboguer les bibliothèques aussi
        },
        {
            "name": "Python: Debug Tests",
            "type": "python",
            "request": "launch",
            "module": "pytest",
            "args": ["-v"],
            "console": "integratedTerminal"
        }
    ]
}
```

### Breakpoints visuels

- Cliquez dans la marge à gauche du numéro de ligne
- Breakpoint conditionnel : clic droit > "Add Conditional Breakpoint"
- Logpoint : affiche un message sans arrêter l'exécution

### Raccourcis clavier

- **F5** : Démarrer le debugging
- **F10** : Step over (next)
- **F11** : Step into
- **Shift+F11** : Step out
- **F9** : Toggle breakpoint

## 9. Techniques de debugging avancées

### Binary Search Debugging

Pour trouver où un bug a été introduit dans l'historique git :

```bash
git bisect start
git bisect bad              # Le commit actuel est bugué
git bisect good <commit>    # Ce commit était OK
# Git va tester les commits entre les deux
# Vous testez et dites "git bisect good" ou "git bisect bad"
```

### Rubber Duck Debugging

Expliquer votre code ligne par ligne à un canard en plastique (ou un collègue) :

1. Préparez-vous à expliquer le problème
2. Expliquez le code ligne par ligne
3. Souvent, vous trouvez le bug en l'expliquant !

### Logging stratégique

Plutôt que des `print()`, utilisez le module `logging` avec différents niveaux :

```python
import logging
logging.basicConfig(level=logging.DEBUG)

def fonction():
    logging.debug("Entrée dans fonction()")
    # ...
    logging.debug(f"Valeur de x: {x}")
```

In [ ]:
import logging

# Configuration du logging pour debugging
logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

logger = logging.getLogger(__name__)

def calcul_complexe(a, b):
    logger.debug(f"Calcul avec a={a}, b={b}")
    resultat = a * 2 + b * 3
    logger.debug(f"Résultat intermédiaire: {resultat}")
    return resultat

calcul_complexe(5, 10)

## Pièges courants

### 1. Oublier de retirer les breakpoint()

```python
# ❌ MAUVAIS : Laisser breakpoint() en production
def fonction_production():
    breakpoint()  # Oups, le serveur va se bloquer !
    return "résultat"

# ✅ BON : Utiliser une variable d'environnement
import os
DEBUG = os.getenv('DEBUG', 'false').lower() == 'true'

def fonction_production():
    if DEBUG:
        breakpoint()
    return "résultat"

# Ou désactiver globalement : PYTHONBREAKPOINT=0
```

### 2. Debugger du code asynchrone

```python
# ⚠️ ATTENTION : pdb ne fonctionne pas bien avec asyncio
import asyncio

async def fonction_async():
    # breakpoint()  # Peut causer des problèmes
    await asyncio.sleep(1)

# ✅ Utiliser aioconsole ou pudb pour le code async
# pip install aioconsole
```

### 3. Debugger avec des effets de bord

```python
# ⚠️ ATTENTION : Éviter de modifier l'état dans le debugger
def traiter(data):
    breakpoint()
    # Dans pdb, si vous faites : data.append(999)
    # Vous modifiez les données pendant le debug !
    return data

# Utilisez plutôt des copies pour tester
```

### 4. Ne pas utiliser l'historique pdb

```python
# Dans pdb, vous pouvez utiliser les flèches haut/bas
# pour naviguer dans l'historique des commandes
# Et Tab pour l'autocomplétion !
```

## Mini-Exercices

### Exercice 1 : Debugger une fonction buggée

Trouvez et corrigez le(s) bug(s) dans cette fonction :

In [ ]:
def trouver_nombres_premiers(n):
    """Retourne tous les nombres premiers jusqu'à n."""
    premiers = []
    for num in range(2, n):
        est_premier = True
        for diviseur in range(2, num):
            if num % diviseur == 0:
                est_premier = False
        if est_premier:
            premiers.append(num)
    return premiers

# Test
resultat = trouver_nombres_premiers(20)
print(f"Nombres premiers jusqu'à 20: {resultat}")
# Devrait retourner: [2, 3, 5, 7, 11, 13, 17, 19]

# Question: Est-ce que cette fonction fonctionne correctement?
# Si non, utilisez des techniques de debugging pour trouver le problème

In [ ]:
# Votre solution corrigée ici


### Exercice 2 : Utiliser pdb pour trouver un bug

Créez un script qui utilise `breakpoint()` pour déboguer cette fonction :

In [ ]:
%%writefile exercice2.py
#!/usr/bin/env python3
def calculer_statistiques(donnees):
    """Calcule min, max, moyenne d'une liste."""
    # Ajoutez un breakpoint ici pour inspecter les données
    
    minimum = min(donnees)
    maximum = max(donnees)
    moyenne = sum(donnees) / len(donnees)
    
    return {
        'min': minimum,
        'max': maximum,
        'moyenne': moyenne
    }

if __name__ == "__main__":
    data = [10, 20, 15, 30, 25]
    stats = calculer_statistiques(data)
    print(stats)

# Instructions:
# 1. Ajoutez breakpoint() au bon endroit
# 2. Exécutez avec: python exercice2.py
# 3. Dans pdb, utilisez 'p donnees', 'n', 'p minimum', etc.
# 4. Vérifiez que les calculs sont corrects

### Exercice 3 : Post-mortem debugging

Créez un script avec gestion d'erreur et post-mortem debugging :

In [ ]:
%%writefile exercice3.py
#!/usr/bin/env python3
import pdb
import sys

def diviser_liste(liste, diviseur):
    """Divise tous les éléments d'une liste par un diviseur."""
    resultats = []
    for element in liste:
        resultat = element / diviseur
        resultats.append(resultat)
    return resultats

def main():
    nombres = [10, 20, 30, 40]
    diviseur = 0  # Bug intentionnel
    
    # TODO: Ajoutez un try/except avec pdb.post_mortem()
    resultats = diviser_liste(nombres, diviseur)
    print(f"Résultats: {resultats}")

if __name__ == "__main__":
    main()

# Instructions:
# 1. Ajoutez un bloc try/except
# 2. En cas d'exception, lancez pdb.post_mortem()
# 3. Exécutez et debuggez pour trouver la ligne qui cause l'erreur

## Solutions

### Solution Exercice 1

In [ ]:
def trouver_nombres_premiers_v2(n):
    """
    Retourne tous les nombres premiers jusqu'à n.
    
    Bug corrigé: La boucle interne testait tous les diviseurs jusqu'à num-1,
    ce qui est inefficace. On peut s'arrêter à sqrt(num).
    De plus, on peut optimiser en ne testant que les diviseurs impairs.
    """
    import math
    
    if n < 2:
        return []
    
    premiers = [2]  # 2 est le seul nombre premier pair
    
    # Tester seulement les nombres impairs
    for num in range(3, n, 2):
        est_premier = True
        # Optimisation: tester seulement jusqu'à sqrt(num)
        for diviseur in range(3, int(math.sqrt(num)) + 1, 2):
            if num % diviseur == 0:
                est_premier = False
                break  # Optimisation: sortir dès qu'on trouve un diviseur
        if est_premier:
            premiers.append(num)
    
    return premiers

# Test
resultat = trouver_nombres_premiers_v2(20)
print(f"Nombres premiers jusqu'à 20: {resultat}")
assert resultat == [2, 3, 5, 7, 11, 13, 17, 19], "Erreur dans le calcul"

### Solution Exercice 2

In [ ]:
%%writefile exercice2_solution.py
#!/usr/bin/env python3
def calculer_statistiques(donnees):
    """Calcule min, max, moyenne d'une liste."""
    # Point d'arrêt pour inspecter les données
    breakpoint()
    
    minimum = min(donnees)
    maximum = max(donnees)
    moyenne = sum(donnees) / len(donnees)
    
    # Point d'arrêt pour vérifier les résultats
    breakpoint()
    
    return {
        'min': minimum,
        'max': maximum,
        'moyenne': moyenne
    }

if __name__ == "__main__":
    data = [10, 20, 15, 30, 25]
    stats = calculer_statistiques(data)
    print(stats)

# Commandes pdb à utiliser:
# p donnees     -> [10, 20, 15, 30, 25]
# n             -> Ligne suivante
# p minimum     -> 10
# p maximum     -> 30
# p moyenne     -> 20.0
# c             -> Continuer

### Solution Exercice 3

In [ ]:
%%writefile exercice3_solution.py
#!/usr/bin/env python3
import pdb
import sys

def diviser_liste(liste, diviseur):
    """Divise tous les éléments d'une liste par un diviseur."""
    resultats = []
    for element in liste:
        resultat = element / diviseur
        resultats.append(resultat)
    return resultats

def main():
    nombres = [10, 20, 30, 40]
    diviseur = 0  # Bug intentionnel
    
    try:
        resultats = diviser_liste(nombres, diviseur)
        print(f"Résultats: {resultats}")
    except Exception as e:
        print(f"Erreur détectée: {e}")
        print("Lancement du debugger post-mortem...")
        # Récupérer le traceback et lancer pdb
        type, value, traceback = sys.exc_info()
        pdb.post_mortem(traceback)

if __name__ == "__main__":
    main()

# Dans pdb:
# l             -> Voir le code autour de l'erreur
# p element     -> Voir la valeur de element
# p diviseur    -> 0 (c'est le problème !)
# up            -> Remonter dans la stack
# p nombres     -> [10, 20, 30, 40]
# q             -> Quitter